# Solving Multiple Tasks in a Sequence of Async Chats

This notebook showcases how to use the new chat interface of conversational agents in AutoGen: a_initiate_chats, to conduct a series of tasks. Similar to "notebook/agentchat_microsoft_fabric.ipynb", this new interface allows one to pass multiple tasks and their corresponding dedicated agents and execute concurrently. Depending on the prerequisite task(s), the tasks will be solved concurrently, with the summaries from prerequisite task(s) provided to subsequent tasks as context, if the `summary_method` argument is specified.

\:\:\:info Requirements

Install `autogen-agentchat`:
```bash
pip install autogen-agentchat~=0.2
```

For more information, please refer to the [installation guide](/docs/installation/).

\:\:\:

In [1]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list}

\:\:\:tip

Learn more about the various ways to configure LLM endpoints [here](/docs/topics/llm_configuration).

\:\:\:

### Example Tasks
Below are four example tasks, with each task being a string of text describing the request. The completion of later tasks requires or benefits from the results of prerequisite tasks.


In [2]:
financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
    """Plot a graph comparing the stock prices over the past month.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Scenario 1: Solve the tasks with a series of chats

The `initiate_chats` interface can take a list of dictionaries as inputs. Each dictionary preserves the following fields: 
- `message`: is a string of text (typically a message containing the task);
- `recipient`: a conversable agent dedicated for the task;
- `summary_method`: A string specifying the method to get a summary from the chat. Currently supported choices include `last_msg`, which takes the last message from the chat history as the summary, and `reflection_with_llm`, which uses an LLM call to reflect on the chat history and summarize a takeaway;
- `summary_prompt`: A string specifying how to instruct an LLM-backed agent (either the recipient or the sender in the chat) to reflect on the chat history and derive a summary. If not otherwise specified, a default prompt will be used when `summary_method` is `reflection_with_llm`.
"Summarize the takeaway from the conversation. Do not add any introductory phrases. If the intended request is NOT properly addressed, please point it out."
- `carryover`: A string or a list of string to specify additional context to be used in the chat. With `initiate_chats`, summary from previous chats will be added as carryover. They will be appended after the carryover provided by the user.

In [3]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

In [4]:
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = await user.a_initiate_chats(  # noqa: F704
    [
        {
            "chat_id": 1,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 2,
            "prerequisites": [1],
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 3,
            "prerequisites": [1],
            "recipient": financial_assistant,
            "message": financial_tasks[2],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {"chat_id": 4, "prerequisites": [1, 2, 3], "recipient": writer, "silent": False, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Starting a new chat....

Message:
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

Carryover: 


********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To obtain current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), and calculate their performance over the past month in terms of percentage change, we can use the Yahoo Finance API through the `yfinance` Python package. This process involves the following steps:

1. Install the `yfinance` package (if not already installed).
2. Use `yfinance` to fetch the curr

/workspaces/autogen/autogen/agentchat/chat.py:42: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(



>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is sh)...
User (to Financial_assistant):

exitcode: 0 (execution succeeded)
Code output: 
NVDA (NVIDIA Corp) current price: $878.37, Performance over past month: 20.97%
TSLA (Tesla, Inc.) current price: $163.57, Performance over past month: -18.19%

Defaulting to user installation because normal site-packages is not writeable


--------------------------------------------------------------------------------
Financial_assistant (to User):

The code execution was successful, and here are the results based on the output:

- **NVIDIA Corporation (NVDA)** has a current stock price of **$878.37**. Over the past month, NVDA has experienced a performance increase of **20.97%**.
- **Tesla, Inc. (TSLA)** has a current stock price of **$163.57**. Over the past month, TSLA has experienced a performance decrease of **-18.19%**.

These figures provide a snapshot of how both companies have performed in the stock market over the last month, reflectin

#### Check chat results
The `initiate_chat` method returns a `ChatResult` object, which is a dataclass object storing information about the chat. Currently, it includes the following attributes:

- `chat_history`: a list of chat history.
- `summary`: a string of chat summary. A summary is only available if a summary_method is provided when initiating the chat.
- `cost`: a tuple of (total_cost, total_actual_cost), where total_cost is a dictionary of cost information, and total_actual_cost is a dictionary of information on the actual incurred cost with cache.
- `human_input`: a list of strings of human inputs solicited during the chat. (Note that since we are setting `human_input_mode` to `NEVER` in this notebook, this list is always empty.)

In [5]:
for i, chat_res in chat_results.items():
    print(f"*****{i}th chat*******:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

*****1th chat*******:
NVIDIA Corporation (NVDA) experienced a stock price increase of 20.97% over the past month, with a current price of $878.37. Tesla, Inc. (TSLA) saw a decrease of 18.19% in the same period, with its current stock price at $163.57.
Human input in the middle: []
Conversation cost:  ({'total_cost': 0.060149999999999995, 'gpt-4-0125-preview': {'cost': 0.060149999999999995, 'prompt_tokens': 3963, 'completion_tokens': 684, 'total_tokens': 4647}}, {'total_cost': 0.04074, 'gpt-4-0125-preview': {'cost': 0.04074, 'prompt_tokens': 3462, 'completion_tokens': 204, 'total_tokens': 3666}})



*****2th chat*******:
Investigating the reasons behind the stock performance of NVIDIA Corporation and Tesla, Inc. requires analyzing recent news, market trends, financial performance, investor sentiment, and industry impacts. For NVIDIA, increased stock prices may relate to demand for GPUs, advances in gaming, data centers, and AI development. Tesla's stock decrease could stem from producti

### Scenario 2: With human inputs revising tasks in the middle

Since AutoGen agents support soliciting human inputs during a chat if `human_input_mode` is specificed properly, the actual task might be revised in the middle of a chat. 

The example below showcases that even if a task is revised in the middle (for the first task, the human user requests to get Microsoft's stock price information as well, in addition to NVDA and TSLA), the `reflection_with_llm`` summary method can still capture it, as it reflects on the whole conversation instead of just the original request.

In [6]:
user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",  # ask human for input at each step
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

chat_results = await user.a_initiate_chats(  # noqa: F704
    [
        {
            "chat_id": 1,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 2,
            "prerequisites": [1],
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "silent": True,
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 3,
            "prerequisites": [1],
            "recipient": financial_assistant,
            "message": financial_tasks[2],
            "silent": False,
            "summary_method": "reflection_with_llm",
        },
        {"chat_id": 4, "prerequisites": [1, 2, 3], "recipient": writer, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Starting a new chat....

Message:
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

Carryover: 


********************************************************************************
User (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User):

To obtain current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), and calculate their performance over the past month in terms of percentage change, we can use the Yahoo Finance API through the `yfinance` Python package. This process involves the following steps:

1. Install the `yfinance` package (if not already installed).
2. Use `yfinance` to fetch the curr

/workspaces/autogen/autogen/agentchat/chat.py:42: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


User (to Financial_assistant):

Pls add Microsoft as well

--------------------------------------------------------------------------------
Financial_assistant (to User):

To include Microsoft (MSFT) in the analysis alongside NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), we can modify the provided Python script to fetch and calculate the performance for MSFT as well. The revised script will now include Microsoft's current stock price and its performance over the past month in terms of percentage change. Please execute the updated Python code below:

```python
# filename: stock_info_updated.py
import yfinance as yf
from datetime import datetime, timedelta

def stock_performance(ticker):
    # Fetching data for the past month
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30)  # Approximate for the past month
    
    # Fetch historical data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Current and beginning of month prices
    

#### Check chat results

In [7]:
for i, chat_res in chat_results.items():
    print(f"*****{i}th chat*******:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

*****1th chat*******:
The current stock prices and the percentage changes over the past month for NVIDIA Corporation (NVDA), Tesla, Inc. (TSLA), and Microsoft Corp (MSFT) are as follows:

- NVDA: Current price $878.37, with a 20.97% increase.
- TSLA: Current price $163.57, with an 18.19% decrease.
- MSFT: Current price $416.42, with a 3.06% increase.

NVDA showed significant growth, MSFT experienced moderate growth, while TSLA saw a decline in its stock price over the last month.
Human input in the middle: ['Pls add Microsoft as well', '', '', '']
Conversation cost:  ({'total_cost': 0.18808000000000002, 'gpt-4-0125-preview': {'cost': 0.18808000000000002, 'prompt_tokens': 10732, 'completion_tokens': 2692, 'total_tokens': 13424}}, {'total_cost': 0.12617, 'gpt-4-0125-preview': {'cost': 0.12617, 'prompt_tokens': 8735, 'completion_tokens': 1294, 'total_tokens': 10029}})



*****2th chat*******:
NVIDIA Corporation (NVDA) showed significant growth possibly due to strong demand for its GPUs in

### Scenario 3: Solve the tasks with a series of chats involving group chat

In [8]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
)

writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    Reply "TERMINATE" in the end when everything is done.
    """,
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.
    Reply "TERMINATE" in the end when everything is done.
    """,
    llm_config={"config_list": config_list},
)

groupchat_1 = autogen.GroupChat(agents=[user_proxy, research_assistant, critic], messages=[], max_round=50)

groupchat_2 = autogen.GroupChat(agents=[user_proxy, writer, critic], messages=[], max_round=50)

manager_1 = autogen.GroupChatManager(
    groupchat=groupchat_1,
    name="Research_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)
manager_2 = autogen.GroupChatManager(
    groupchat=groupchat_2,
    name="Writing_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
await user.a_initiate_chats(  # noqa: F704
    [
        {"chat_id": 1, "recipient": research_assistant, "message": financial_tasks[0], "summary_method": "last_msg"},
        {
            "chat_id": 2,
            "prerequisites": [1],
            "recipient": manager_1,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "chat_id": 3,
            "prerequisites": [1],
            "recipient": manager_1,
            "message": financial_tasks[2],
            "summary_method": "reflection_with_llm",
        },
        {"chat_id": 4, "prerequisites": [1, 2, 3], "recipient": manager_2, "message": writing_tasks[0]},
    ]
)


********************************************************************************
Starting a new chat....

Message:
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

Carryover: 


********************************************************************************
User (to Researcher):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Researcher (to User):

To obtain current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), and calculate their performance over the past month in terms of percentage change, we can use the Yahoo Finance API through the `yfinance` Python package. This process involves the following steps:

1. Install the `yfinance` package (if not already installed).
2. Use `yfinance` to fetch the current stock prices o

{1: ChatResult(chat_id=1, chat_history=[{'content': 'What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?', 'role': 'assistant'}, {'content': 'To obtain current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), and calculate their performance over the past month in terms of percentage change, we can use the Yahoo Finance API through the `yfinance` Python package. This process involves the following steps:\n\n1. Install the `yfinance` package (if not already installed).\n2. Use `yfinance` to fetch the current stock prices of NVDA and TSLA.\n3. Fetch historical data for the past month for both stocks.\n4. Calculate the percentage change in the stock prices over the last month based on the historical data.\n\nPlease execute the following Python code to perform these tasks:\n\n```python\n# filename: stock_info.py\nimport yfinance as yf\nfrom datetime import datetime, timedelta\n\ndef stock_perform